# Document Clustering & Analysis using Custom K-Means
This **Jupyter Notebook** looks a subset of the Newsgroup dataset. The **subset** for this dataset includes 2,500 documents, each belonging to one of **5 categories** which are: Windows (0), Crypt (1), Christian (2), Hockey (3), Forsale (4). The documents are represented by 9328 terms (stems). 

The **vocabulary** for the dataset is given in the file "terms.txt" and the **term-by-document** matrix is given in "matrix.txt". The actual category labels for the document is provided in the file "classes.txt". The **goal** of this lab is to perform clustering on the documents and compare the clusters to the actual categories.

### Importing Libraries
We start by importing the following libraries:
- **Pandas:** a powerful open-source data analysis and manipulation library for Python. It provides data structures and functions for efficiently working with structured data
- **NumPy:** a open source Python library that's widely used in science and engineering. The NumPy library contains multidimensional array data structures, such as the homogeneous, N-dimensional ndarray
- **Sklearn:**  a free and open-source machine learning library for the Python programming language
- **kMeans:** this is a python script which contains the functions that are going to be used to calculate distances and etc

In [1]:
import sys
import os
import pandas as pd
import numpy as np
from kMeans import *
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import completeness_score, homogeneity_score

## 1 |  Cosine Distance Function
We start off by creating the cosine distance function. We create our own distance function, where instead of using Euclidean distance, we use Cosine Similarity. This is the distance function that will be used to pass to the KMeans function in the Python Script. We use a written version that computes the Cosine Similarity between two n-dimensional vectors and returns the inverse as the distance between the vectors. 

### Changes to the KMeans.py File
Here is the following change that was made to the `kMeans.py` file where the Cosine Distance function was added with the following logic:
```
def distCosine(vecA, vecB):
    vector_A = np.array(vecA)
    vector_B = np.array(vecB)

    dot_product = np.dot(vector_A, vector_B)
    magnitude_A = np.linalg.norm(vector_A)
    magnitude_B = np.linalg.norm(vector_B)

    if magnitude_A == 0 or magnitude_B == 0:
        return 1.0

    cosine_similarity = dot_product / (magnitude_A * magnitude_B)
    cosine_distance = 1 - cosine_similarity

    return cosine_distance
```

## 2 | Data Preprocessing

Next up, we perform the data preprocessing. We start off by loading the data sets and then perform the following:
- [x] convert the data into a numpy array and and then transpose it to get the term-document matrix 
- [x] split the data set into training and testing data sets, where 20% of the dataset is saved for testning
- [x] perform the tf-idf transformation 

### The Matrix Data from `matrix.txt`
We start off by loading the matrix.txt using `np.loadtxt()` and the delimiter is set to `,` because all the numbers are separated by commas. Once the numpy arrays have been loaded into the variable `data_matrix`, what we end up with the term-document matrix. Since we're going to be perform clustering, we perform the transpose by using `.T` which will give us the document-term matrix

In [2]:
# loading the matrix data and transposing it
data_matrix = np.loadtxt("matrix.txt", delimiter=",")
data_matrix = data_matrix.T

### Splitting of the Dataset
We then use the document-term matrix to split the data into training and testing dataset. 20% of the dataset is reserved for training and 80% for training. We use `random_state` to ensure that we're performing a randomized split of the dataset.

In [3]:
# splitting the dataset into training and testing dataset
train_data, test_data = train_test_split(data_matrix, test_size=0.2, random_state=99)

### The TF-IDF Transformation
Lastly, we initialize the Tfidf transformer which is used to compute the TF-IDF representation of term-document matrices. We apply two tranformations, which is the term frequency and the inverse document frequency. We perform this on both the testing and training data.

In [4]:
# performing the TF-IDF transformation
tfidf_transformer = TfidfTransformer()
train_data_tfidf = tfidf_transformer.fit_transform(train_data).toarray()
test_data_tfidf = tfidf_transformer.transform(test_data).toarray()

## 3 | Performing KMeans Clustering 

Next up, we peform the KMeans Clustering on the tranformed training data. This is to conduct a analysis of the clusters by examining the top features in each cluster to identify patterns in the data. 

For this, we use a for loop to display the top terms in each cluster, sorting them by their average TF-IDF weight from the cluster centroid. We also use the for loop to display the data about each cluster.

### Opening the Text File
We open the `terms.txt` file in read mode, where the program reads its contents. We then use list comprehension iterate over each line in the file object and create a list of words and store them in the variable called `vocabulary`.

In [5]:
with open('terms.txt', 'r') as file:
    vocabulary = [line.strip() for line in file]

### Different Vaues of K for Performing Clustering
Starting off with a for-loop, we iterate through the K-values of 4 and 5. We run the `kMeans.py` script and suppress the print statements by redirecting the the standard output `stdout` to `os.devnull`. We then perform the k-means clustering and show the results of the clustering. Here are the next following steps:

- [x] Iterate though the clusters corresponding to the values of k and calculate the centroids
- [x] Create print statements that show the mean TF-IDF weights of that term, cluster DF count and cluster DF percentage

In [6]:
# for loop through the values of k
for k in range(4, 6):
    print(f"Running K-means with k = {k}")
    
    # suppression of print statements
    stdout = sys.stdout 
    sys.stdout = open(os.devnull, 'w')
    
    # running the K-means clustering algorithm using the kMeans.py file
    centroids, cluster_assment = kMeans(train_data_tfidf, k, distMeas=distCosine, createCent=randCent)
    
    # end of suppression as we restore the standard output
    sys.stdout.close()
    sys.stdout = stdout

    # extracting the number of documents and features from the shape of the matrix
    n_docs, n_terms = train_data_tfidf.shape
    
    # to display the top 5 items
    top_n = 5

    # iterating through the clusters
    for cluster in range(k):
        
        # retrieving the indices, tf-idf vectors of documents and number of documents
        cluster_indices = np.nonzero(cluster_assment[:, 0] == cluster)[0]
        cluster_docs = train_data_tfidf[cluster_indices]
        cluster_size = len(cluster_docs)

        # calculating the mean tf-idf vector and represents the centroid of the cluster
        avg_tfidf = centroids[cluster]

        # identifying the top terms in the cluster by sorting the term indices based on tf-idf weights
        top_indices = np.argsort(avg_tfidf)[::-1][:top_n]
        top_terms = [(vocabulary[i], avg_tfidf[i]) for i in top_indices]

        # printing the top 5 terms in each cluster as well as other details
        print(f"Cluster {cluster + 1} Size = {cluster_size}")
        print(f"{'Term':<15}{'Mean TF-IDF':<12}{'DF':<6}{'% of Docs':<10}")
        print("-" * 45)

        # printing the index of the term, how many documents in the cluster contain the term and percentage
        for term, weight in top_terms:
            term_index = vocabulary.index(term)
            term_doc_count = np.sum(cluster_docs[:, term_index] > 0)
            term_doc_percentage = (term_doc_count / cluster_size) * 100
            print(f"{term:<15}{weight:<12.2f}{term_doc_count:<6}{term_doc_percentage:<10.2f}")

        print("\n")

Running K-means with k = 4
Cluster 1 Size = 793
Term           Mean TF-IDF DF    % of Docs 
---------------------------------------------
window         0.06        302   38.08     
sale           0.04        236   29.76     
file           0.03        161   20.30     
subject        0.03        793   100.00    
driver         0.02        95    11.98     


Cluster 2 Size = 399
Term           Mean TF-IDF DF    % of Docs 
---------------------------------------------
game           0.07        206   51.63     
team           0.05        184   46.12     
plai           0.04        162   40.60     
hockei         0.04        171   42.86     
go             0.03        176   44.11     


Cluster 3 Size = 408
Term           Mean TF-IDF DF    % of Docs 
---------------------------------------------
god            0.07        223   54.66     
christian      0.05        182   44.61     
sin            0.03        84    20.59     
jesu           0.03        111   27.21     
church         0.03 

### Discussion
It can be seen that the best clustering result is when `k=5`. This is because when `k=5`, we can see that the clusters have produced easy to interpret theme or a common topic compared to when `k=4`.

When we look at the clusters, for example at `Cluster 2`, it can be seen that the top terms are `god`, `christian`, `jesus`, `sin` and `church`. This means that the cluster is about topics related to Christianity. Other similar themes are discovered in other clusters, for example in `Cluster 1`, the topic might be about Cryptography because the terms are `encrypt` or `kei`.

## 4 | Evaluating Cluster Quality
Now, we evaluate the quality of the clusters. Using the clusters we produced, we assess how well the clusters match the actual categories by calculating the Completeness and Homogeneity Scores. Here are the following things to do:
- [x] determining the representative label based on a majority vote of the true labels on the training documents
- [x] calculating the completeness and homoegeneity scores, where completeness indicates whether all documents as assigned to the same cluster, and homogeneity determines whether each cluster contains only documents that belong to a single cateogory
- [x] we perform this analysis using the best clustering result that was identified in the previous part 

### Loading Category Labels
Here, we load the `classes.txt` file which contains all the labels, and the split them into test and training data, where 20% of the data is dedicated towards testing and 80% towards training. We use `random_state` to ensure the splitting is randomized.

In [7]:
# retrieving the true cateogry labels
true_labels = np.loadtxt('classes.txt', dtype=int, comments='%', usecols=1)

# splitting the labels into training and testing data
train_labels, test_labels = train_test_split(true_labels, test_size=0.2, random_state=99)

### Mapping Clusters to Representative Label
Next, we initailize a dictionary to map each cluster to its representative label. For each cluster, we identify the documents assigned to that cluster, retrieve the tue label, and determine the most frequent label, which is then assigned to that cluster.

In [8]:
# intializing a dictionary
cluster_to_label = {}

# iterating using for loop through each cluster
for cluster in range(k):
    
    # retreiving the indices of the cluster
    cluster_indices = np.nonzero(cluster_assment[:, 0] == cluster)[0]
    
    # retreiving the true labels
    cluster_labels = train_labels[cluster_indices]
    
    # determining the most common label
    most_common_label = Counter(cluster_labels).most_common(1)[0][0]
    cluster_to_label[cluster] = most_common_label

In [9]:
# mapping the cluster assignments to the label
predicted_labels = np.array([cluster_to_label[int(cluster)] for cluster in cluster_assment[:, 0]])

### Computing the Completeness & Homogeneity Scores
Lastly, we compute the completeness and homogeneity scores using the training labels and the predicted labels, and then display the results!

In [10]:
# Compute scores
completeness = completeness_score(train_labels, predicted_labels)
homogeneity = homogeneity_score(train_labels, predicted_labels)

print(f"Completeness Score: {completeness:.4f}")
print(f"Homogeneity Score: {homogeneity:.4f}")


Completeness Score: 0.8493
Homogeneity Score: 0.8492


## 5 | Classifying Test Documents
Using the final cluster centroids from the best K-means clustering, we classify each document in the 20% test set by assinging it to the closest cluster based on the Cosine similarity. Here are the following things to do:
- [x] Calculating the cosine similarity for each document in the test set
- [x] Assinging cluster labels to test documents, where for each document, we assign it to the cluster with the highest cosine similarity score
- [x] displaying the results of the classification by showinf the assigned cluster label and the cosine similairty score

### Calculating Cosine Simialirty & Assignin Cluster Labels
We start off here by classifying each document in the test set by calculating the cosine distance between the documents TF-IDF vector and each cluster centroid. The document is then assigned to the cluster with the smallest distance, which where the cosine similarity distance calculation comes into play. The smallest distance would be the highest cosine similarity.

In [11]:
# storing document classifications
test_results = []

# classifying each document
for idx, doc_vector in enumerate(test_data_tfidf):
    
    # computing cosine distances
    distances = [distCosine(doc_vector, centroid) for centroid in centroids]
    
    # finding the closes centroid
    best_cluster = np.argmin(distances)
    best_similarity = 1 - distances[best_cluster]  # Convert distance back to similarity
    
    # mapping the cluster to its labels
    pseudo_label = cluster_to_label[best_cluster]
    
    # storing the results
    test_results.append((idx, best_cluster, pseudo_label, best_similarity))

### Displaying the Results
Lastly, for each test document, we output the assigned cluster label and the cosine similarity score, as well the ID of the document.

In [12]:
# displaying the results 
print("Test Document Classification Results:")
print("Doc ID | Assigned Cluster | Label | Cosine Similarity")
for doc_id, cluster, label, similarity in test_results:
    print(f"{doc_id:6} | {cluster:16} | {label:12} | {similarity:.4f}")

Test Document Classification Results:
Doc ID | Assigned Cluster | Label | Cosine Similarity
     0 |                1 |            1 | 0.3038
     1 |                1 |            1 | 0.1477
     2 |                0 |            0 | 0.2263
     3 |                1 |            1 | 0.1487
     4 |                3 |            2 | 0.3291
     5 |                2 |            3 | 0.3100
     6 |                3 |            2 | 0.3528
     7 |                3 |            2 | 0.1691
     8 |                2 |            3 | 0.2375
     9 |                3 |            2 | 0.3162
    10 |                2 |            3 | 0.2781
    11 |                4 |            4 | 0.0574
    12 |                2 |            3 | 0.2731
    13 |                2 |            3 | 0.2118
    14 |                3 |            2 | 0.3059
    15 |                3 |            2 | 0.0761
    16 |                2 |            3 | 0.1913
    17 |                3 |            2 | 0.3217
    18 |